#Load Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy
import numpy as np

import os
import time
import random

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#Load CSV File to train the models on

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

path = "/content/drive/My Drive/project/"
training_dataset = pd.read_csv(path + 'em.csv')

training_dataset.columns = ['nfr','mono','di','tri']
training_dataset.head()

,nfr,mono,di,tri
0,2.148752,0.190306,2.152600,0.088854
1,2.148752,0.190306,2.152600,0.088854
2,2.148752,0.190306,2.152600,0.088854
3,2.148752,0.190306,2.152600,0.088854
4,1.222059,1.638257,2.155221,0.088854


In [ ]:
from time import time
import numpy as np
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

##Estimate State Labels by using K-Means Directly

In [ ]:
n_samples, n_features = training_dataset.shape
n_classes = 3
estimator = KMeans(init='k-means++', n_clusters=n_classes, n_init=10)
estimator.fit(training_dataset)
labels = estimator.predict(training_dataset)

In [ ]:
estimator.cluster_centers_

array([[0.19730298, 0.08590303, 0.32614811, 0.05594202],
       [1.2451336 , 2.93929402, 2.42497628, 0.32360281],
       [1.08004871, 2.63965522, 0.22601982, 0.05596694]])

In [ ]:
labels = pd.DataFrame(labels)
labels.head()

,0
0,0
1,0
2,0
3,0
4,1


In [ ]:
nfr_values = np.array([estimator.cluster_centers_[i][0] for i in range(3)])
index_C = np.argmax(nfr_values)
index_B = np.argmin(nfr_values)
index_L = 3 - index_C - index_B
print(nfr_values)
print(index_L)
print(index_B)
print(index_C)


[0.19730298 1.2451336  1.08004871]
2
0
1


In [ ]:
labels = labels.replace({index_C:'C', index_B:'B', index_L:'L'})
# labels = pd.get_dummies(labels[labels.columns[0]])
labels.head()

,0
0,A
1,A
2,A
3,A
4,B


In [ ]:
training_dataset.head()
from torch.utils.data import DataLoader

#Univariate LSTM

##Training LSTM Models

###Train Univariate LSTM for each of the columns

In [ ]:
training_dataset.head()

''' The user needs to train the LSTM for each of the columns in the dataset.
    This is made possible by setting the to_do value to the particular column name.
    For example, here to_do = 'nfr' trains an LSTM over the nfr column values.
    If you want to train an LSTM for each of the columns, simply change the value of to_do (uncomment the other line and comment the current one)
'''

to_do = 'nfr'
# to_do = 'mono'
# to_do = 'di'
# to_do = 'tri'

from sklearn.preprocessing import MinMaxScaler
price = training_dataset[[to_do]]

In [ ]:
price.head()

,tri
0,0.088854
1,0.088854
2,0.088854
3,0.088854
4,0.088854


###Split into Train and Validation Set (Default 80-20 ratio) and Create DataLoaders

In [ ]:
''' Change the Train-Validation split ratio here '''
ratio = 0.2

def split_data(stock, lookback):
    data_raw = stock.to_numpy() # convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(ratio*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

''' Set the context window size. Default is a context size of 20 samples.
    This determines how much of the previous data points you are exposing to the LSTM Model.
    E.g., in this case, you expose 20 sequential samples and then predict the 21st value.
'''

lookback = 20 # choose sequence length

x_train, y_train, x_test, y_test = split_data(price, lookback)

print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (801502, 19, 1)
y_train.shape =  (801502, 1)
x_test.shape =  (200375, 19, 1)
y_test.shape =  (200375, 1)


In [ ]:
# make training and test sets in torch
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

In [ ]:
n_steps = lookback-1
batch_size = 1606
#n_iters = 3000
num_epochs = 50 #n_iters / (len(train_X) / batch_size)

train = torch.utils.data.TensorDataset(x_train,y_train)
test = torch.utils.data.TensorDataset(x_test,y_test)


train_loader = torch.utils.data.DataLoader(dataset=train, 
                                           batch_size=batch_size, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test, 
                                          batch_size=batch_size, 
                                          shuffle=False)

###Define Model Architecture + Optimiser + Loss Functions



In [ ]:
# Build model
#####################
input_dim = 1
hidden_dim = 32
num_layers = 2 
output_dim = 1


# Here we define our model as a class
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # Building your LSTM
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().cuda()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().cuda()

        # One time step
        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :])
        # out.size() --> 100, 10
        return out
    
'''Location to save best performing model based on Validation Set Performance'''

model_dir = "/content/drive/My Drive/LSTM/Uni"

model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).cuda()

loss_fn = torch.nn.MSELoss(size_average=True)

optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

LSTM(
  (lstm): LSTM(1, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)
10
torch.Size([128, 1])
torch.Size([128, 32])
torch.Size([128])
torch.Size([128])
torch.Size([128, 32])
torch.Size([128, 32])
torch.Size([128])
torch.Size([128])
torch.Size([1, 32])
torch.Size([1])


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


###Train Univariate LSTM Model

In [ ]:
# Train model
#####################

'''Set number of epochs. Default  50'''
num_epochs = 50
train_hist = np.zeros(num_epochs)
test_hist = np.zeros(num_epochs)

# Number of steps to unroll
seq_dim = lookback-1  

best_train_loss = 1e10
best_test_loss = 1e10
best_train_epoch = 0
best_test_epoch = 0


for t in range(num_epochs):

      train_loss_epoch = 0
      test_loss_epoch = 0

      model.train()

      for x, y in train_loader:

          x, y = x.cuda(), y.cuda()

          y_pred  = model(x)

          loss = loss_fn(y_pred, y)

          train_loss_epoch += loss.item()

          optimiser.zero_grad()

          loss.backward()
          
          optimiser.step()

          # if t % 10 == 0 and t !=0:
      print("Train Epoch ", t, " Train MSE: ", train_loss_epoch)
      train_hist[t] = train_loss_epoch

      if (train_loss_epoch < best_train_loss):
          best_train_loss = train_loss_epoch
          best_train_epoch = t

      with torch.no_grad():
        
          for x, y in test_loader:

              x, y = x.cuda(), y.cuda()

              output = model(x) 

              loss = loss_fn(output, y)

              test_loss_epoch += loss.item()

      print("Validation Epoch ", t, "Validation MSE: ", test_loss_epoch)
      test_hist[t] = test_loss_epoch

      if (test_loss_epoch < best_test_loss):
          best_test_loss = test_loss_epoch
          best_test_epoch = t

          torch.save({
                      'multi_lstm': model.state_dict(),
                      'optimizer_state_dict': optimiser.state_dict(),
                      }, os.path.join(model_dir, 'Mono_LSTM-{}-{}-{}-{}.pth'.format(to_do, lookback, best_test_epoch, best_test_loss)))
          
      print(train_loss_epoch, best_train_loss, test_loss_epoch, best_test_loss)


Train Epoch  0  Train MSE:  6.488508228108003
Validation Epoch  0 Validation MSE:  0.8921142063627485
6.488508228108003 6.488508228108003 0.8921142063627485 0.8921142063627485
Train Epoch  1  Train MSE:  2.7026635966905133
Validation Epoch  1 Validation MSE:  0.9002020554398769
2.7026635966905133 2.7026635966905133 0.9002020554398769 0.8921142063627485
Train Epoch  2  Train MSE:  2.245682020430422
Validation Epoch  2 Validation MSE:  0.6956993043568218
2.245682020430422 2.245682020430422 0.6956993043568218 0.6956993043568218
Train Epoch  3  Train MSE:  2.318307038556668
Validation Epoch  3 Validation MSE:  0.5522460303800472
2.318307038556668 2.245682020430422 0.5522460303800472 0.5522460303800472
Train Epoch  4  Train MSE:  1.8749866477119213
Validation Epoch  4 Validation MSE:  0.445939100747637
1.8749866477119213 1.8749866477119213 0.445939100747637 0.445939100747637
Train Epoch  5  Train MSE:  2.1189122545329155
Validation Epoch  5 Validation MSE:  0.539640090790499
2.1189122545329

##Predicting the regression values and state Labels



###Load Best Performing Univariate Models for each of the columns

In [ ]:
model_dir = "/content/drive/My Drive/LSTM"

uni_nfr_model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).cuda()
uni_mono_model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).cuda()
uni_di_model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).cuda()
uni_tri_model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).cuda()


checkpointA = torch.load(os.path.join(model_dir, 'Mono_LSTM-nfr-20-40-7.361742030829191.pth'))
checkpointB = torch.load(os.path.join(model_dir, 'Mono_LSTM-mono-20-38-10.677848100662231.pth'))
checkpointC = torch.load(os.path.join(model_dir, 'Mono_LSTM-di-20-40-3.1974508333951235.pth'))
checkpointD = torch.load(os.path.join(model_dir, 'Mono_LSTM-tri-20-44-0.3832133790061789.pth'))

uni_nfr_model.load_state_dict(checkpointA['multi_lstm'])
uni_mono_model.load_state_dict(checkpointB['multi_lstm'])
uni_di_model.load_state_dict(checkpointC['multi_lstm'])
uni_tri_model.load_state_dict(checkpointD['multi_lstm'])

<All keys matched successfully>

###Load CSV file on which to Predict

In [ ]:
import numpy as np
import pandas as pd

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

path = "/content/drive/My Drive/project/"
training_dataset = pd.read_csv(path + 'split-29.csv')

training_dataset.columns = ['nfr','mono','di','tri']
training_dataset.head()

,nfr,mono,di,tri
0,0.0000,0.0000,0.0000,0.0
1,0.0000,0.0000,0.0000,0.0
2,6.5921,1.1372,0.0001,0.0
3,6.5921,1.1372,0.0001,0.0
4,6.5921,1.1372,0.0001,0.0


###Predict values for each of the columns


In [ ]:
final_data = []

model_list = [uni_nfr_model, uni_mono_model, uni_di_model, uni_tri_model]
col_list = ['nfr', 'mono', 'di', 'tri']

for i in range(len(col_list)):
  to_do = col_list[i]
  model = model_list[i]
  price = training_dataset[[to_do]]

  def pred_data(stock, lookback):
      data_raw = stock.to_numpy() # convert to numpy array
      data = []
      
      # create all possible sequences of length seq_len
      for index in range(len(data_raw) - lookback): 
          data.append(data_raw[index: index + lookback])
      
      data = np.array(data)
      train_set_size = data.shape[0]
      
      x_train = data[:train_set_size,:-1,:]
      y_train = data[:train_set_size,-1,:]
      
      return [x_train, y_train]

  x_pred, y_pred = pred_data(price, lookback)

  x_pred = torch.from_numpy(x_pred).type(torch.Tensor)
  y_pred = torch.from_numpy(y_pred).type(torch.Tensor)

  pred = torch.utils.data.TensorDataset(x_pred,y_pred)

  pred_loader = torch.utils.data.DataLoader(dataset=pred, 
                                            batch_size=batch_size, 
                                            shuffle=False)

  # print(training_dataset.shape)
  # print(x_pred.shape, y_pred.shape)

  predictions = []
  with torch.no_grad():

    for x, y in pred_loader:

            x, y = x.cuda(), y.cuda()

            output = uni_nfr_model(x).view(-1).cpu().numpy()

            predictions.extend(output)
              
  temp = price.iloc[0:lookback].values.flatten()
  predictions = np.append(temp, predictions)

  final_data.append(predictions)
  print(final_data)

(999999, 4)
torch.Size([999979, 19, 1]) torch.Size([999979, 1])
[array([0.        , 0.        , 6.5921    , ..., 5.90757418, 5.99875689,
       5.98339081])]
(999999, 4)
torch.Size([999979, 19, 1]) torch.Size([999979, 1])
[array([0.        , 0.        , 6.5921    , ..., 5.90757418, 5.99875689,
       5.98339081]), array([0.        , 0.        , 1.1372    , ..., 0.63269067, 0.62940854,
       0.63487089])]
(999999, 4)
torch.Size([999979, 19, 1]) torch.Size([999979, 1])
[array([0.        , 0.        , 6.5921    , ..., 5.90757418, 5.99875689,
       5.98339081]), array([0.        , 0.        , 1.1372    , ..., 0.63269067, 0.62940854,
       0.63487089]), array([0.        , 0.        , 0.0001    , ..., 0.02607608, 0.02607608,
       0.02607608])]
(999999, 4)
torch.Size([999979, 19, 1]) torch.Size([999979, 1])
[array([0.        , 0.        , 6.5921    , ..., 5.90757418, 5.99875689,
       5.98339081]), array([0.        , 0.        , 1.1372    , ..., 0.63269067, 0.62940854,
       0.63487089

In [ ]:
temp = []

for i in range(len(final_data[0])):
  temp.append([final_data[0][i],final_data[1][i],final_data[2][i],final_data[3][i]])

df = pd.DataFrame(temp, columns=col_list)

df

,nfr,mono,di,tri
0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000
2,6.592100,1.137200,0.000100,0.000000
3,6.592100,1.137200,0.000100,0.000000
4,6.592100,1.137200,0.000100,0.000000
...,...,...,...,...
999994,5.850145,0.625675,0.026076,0.026076
999995,5.811829,0.621055,0.026076,0.026076
999996,5.907574,0.632691,0.026076,0.026076
999997,5.998757,0.629409,0.026076,0.026076


###Run K-Means to assign State Labels for each row

In [ ]:
n_samples, n_features = df.shape
n_classes = 3
estimator = KMeans(init='k-means++', n_clusters=n_classes, n_init=10)
estimator.fit(df)
labels = estimator.predict(df)
labels = pd.DataFrame(labels)
estimator.cluster_centers_
nfr_values = np.array([estimator.cluster_centers_[i][0] for i in range(3)])
index_C = np.argmax(nfr_values)
index_B = np.argmin(nfr_values)
index_L = 3 - index_C - index_B

labels = labels.replace({index_C:'C', index_B:'B', index_L:'L'})
labels.head()

[497421. 832043. 165377.]
0
2
1


,0
0,B
1,B
2,B
3,B
4,B


###Save the Regression and Label Prediction Files to the drive

In [ ]:
df.to_csv('mono_pred.csv')
!cp mono_pred.csv "drive/My Drive/"

labels.to_csv('mono_labels.csv')
!cp mono_labels.csv "drive/My Drive/"

#Multivariate LSTM

##Training LSTM Models

###Define Model Architecture

In [ ]:
class MV_LSTM(torch.nn.Module):
    def __init__(self,n_features,seq_length):
        super(MV_LSTM, self).__init__()
        self.n_features = n_features
        self.seq_len = seq_length
        self.n_hidden = 20 # number of hidden states
        self.n_layers = 1 # number of LSTM layers (stacked)
    
        self.l_lstm = torch.nn.LSTM(input_size = n_features, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                 batch_first = True)
        # according to pytorch docs LSTM output is 
        # (batch_size,seq_len, num_directions * hidden_size)
        # when considering batch_first = True
        self.l_linear = torch.nn.Linear(self.n_hidden*self.seq_len, 1)
        
    
    def init_hidden(self, batch_size):
        # even with batch_first = True this remains same as docs
        hidden_state = torch.zeros(self.n_layers,batch_size,self.n_hidden).requires_grad_().cuda()
        cell_state = torch.zeros(self.n_layers,batch_size,self.n_hidden).requires_grad_().cuda()
        self.hidden = (hidden_state, cell_state)
    
    
    def forward(self, x):        
        batch_size, seq_len, _ = x.size()
        
        lstm_out, self.hidden = self.l_lstm(x,self.hidden)
        # lstm_out(with batch_first = True) is 
        # (batch_size,seq_len,num_directions * hidden_size)
        # for following linear layer we want to keep batch_size dimension and merge rest       
        # .contiguous() -> solves tensor compatibility error
        x = lstm_out.contiguous().view(batch_size,-1)
        return self.l_linear(x)

In [ ]:
dataset = training_dataset
print(dataset.head)
print(dataset.iloc[0:3, -1])
print(dataset[0:3]["tri"])

dataset = training_dataset.to_numpy()
print(dataset)
# print(dataset.iloc[[0:3],[1,2]])
# dataset.iloc[1:3]

<bound method NDFrame.head of               nfr      mono        di       tri
0        2.148752  0.190306  2.152600  0.088854
1        2.148752  0.190306  2.152600  0.088854
2        2.148752  0.190306  2.152600  0.088854
3        2.148752  0.190306  2.152600  0.088854
4        1.222059  1.638257  2.155221  0.088854
...           ...       ...       ...       ...
1001892  0.000000  0.000000  0.000000  0.000000
1001893  0.000000  0.000000  0.000000  0.000000
1001894  0.000000  0.000000  0.000000  0.000000
1001895  0.000000  0.000000  0.000000  0.000000
1001896  0.000000  0.000000  0.000000  0.000000

[1001897 rows x 4 columns]>
0    0.088854
1    0.088854
2    0.088854
Name: tri, dtype: float64
0    0.088854
1    0.088854
2    0.088854
Name: tri, dtype: float64
[[2.14875224 0.1903058  2.15259968 0.08885376]
 [2.14875224 0.1903058  2.15259968 0.08885376]
 [2.14875224 0.1903058  2.15259968 0.08885376]
 ...
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0

###Process the dataset to adjust for context window.

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps, n_features, to_do):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix + 1 > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :n_features], sequences[end_ix, -mapping[to_do]]
        X.append(seq_x)
        y.append(seq_y)

        # print(X)
        # print(y)

        # break
    return np.array(X), np.array(y)

In [ ]:
from array import array

''' The user needs to train the LSTM for each of the columns in the dataset.
    This is made possible by setting the to_do value to the particular column name.
    For example, here to_do = 'nfr' trains an LSTM over the nfr column values.
    If you want to train an LSTM for each of the columns, simply change the value of to_do (uncomment the other line and comment the current one)
'''

to_do = 'nfr'
# to_do = 'mono'
# to_do = 'di'
# to_do = 'tri'

mapping = {'nfr':0, 'mono':1, 'di':2, 'tri':3}

n_features = 4 # this is number of parallel inputs
n_timesteps = 20 # this is number of timesteps
look_back = 20

# convert dataset into input/output
X, y = split_sequences(dataset, n_timesteps, n_features, to_do)
print(X.shape, y.shape)

(1001877, 20, 4) (1001877,)


###Split into Train and Validation Set (Default 80-20 ratio) and Create DataLoaders

In [ ]:
ratio = 0.2

test_set_size = int(np.round(ratio*X.shape[0]));
train_set_size = X.shape[0] - (test_set_size);

x_train = X[:train_set_size]
y_train = y[:train_set_size]

x_test = X[train_set_size:]
y_test = y[train_set_size:]

x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

print(x_train.shape)
print(y_train.shape)

torch.Size([801502, 20, 4])
torch.Size([801502])


###Define Optimiser + Loss Function



###Train Multivariate LSTM Models

In [ ]:
# create NN
mv_net = MV_LSTM(n_features,n_timesteps).cuda()
criterion = torch.nn.MSELoss() # reduction='sum' created huge loss value
optimizer = torch.optim.Adam(mv_net.parameters(), lr=0.001)

train_episodes = 20
num_epochs = train_episodes
batch_size = 32
mv_net.train()


train = torch.utils.data.TensorDataset(x_train,y_train)
test = torch.utils.data.TensorDataset(x_test,y_test)

train_loader = torch.utils.data.DataLoader(dataset=train, 
                                           batch_size=batch_size, 
                                           shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test, 
                                          batch_size=batch_size, 
                                          shuffle=False)


train_hist = np.zeros(num_epochs)
test_hist = np.zeros(num_epochs)

best_train_loss = 1e10
best_test_loss = 1e10
best_train_epoch = 0
best_test_epoch = 0


for t in range(train_episodes):

    train_loss_epoch = 0
    test_loss_epoch = 0

    for x, y in train_loader:
        x_batch = x.cuda()
        y_batch = y.cuda()
    
        mv_net.init_hidden(x_batch.size(0))

        output = mv_net(x_batch) 
        loss = criterion(output.view(-1), y_batch)  

        train_loss_epoch += loss.item()
        
        loss.backward()
        optimizer.step()        
        optimizer.zero_grad() 

    print("Train Epoch ", t, " Train MSE: ", train_loss_epoch)
    train_hist[t] = train_loss_epoch

    if (train_loss_epoch < best_train_loss):
        best_train_loss = train_loss_epoch
        best_train_epoch = t

    with torch.no_grad():

        for x, y in test_loader:

            x_batch = x.cuda()
            y_batch = y.cuda()
        
            mv_net.init_hidden(x_batch.size(0))

            output = mv_net(x_batch) 

            loss = criterion(output.view(-1), y_batch) 

            test_loss_epoch += loss.item()

        print("Validation Epoch ", t, "Validation MSE: ", test_loss_epoch)
        test_hist[t] = test_loss_epoch

        if (test_loss_epoch < best_test_loss):
            best_test_loss = test_loss_epoch
            best_test_epoch = t

            torch.save({
                        'multi_lstm': mv_net.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        }, os.path.join(model_dir, 'Multi_LSTM-{}-{}-{}-{}.pth'.format(to_do, lookback, best_test_epoch, best_test_loss)))


KeyboardInterrupt: ignored

##Predicting the regression values and State Labels

###Load Best Performing Multivariate Models for each of the columns

In [ ]:
import os
model_dir = "/content/drive/My Drive/LSTM"

# !ls $model_dir
multi_nfr_model = MV_LSTM(n_features,n_timesteps).cuda()
multi_mono_model = MV_LSTM(n_features,n_timesteps).cuda()
multi_di_model = MV_LSTM(n_features,n_timesteps).cuda()
multi_tri_model = MV_LSTM(n_features,n_timesteps).cuda()


checkpointA = torch.load(os.path.join(model_dir, 'Multi_LSTM-nfr-20-0.pth'))
checkpointB = torch.load(os.path.join(model_dir, 'Multi_LSTM-mono-20-0.pth'))
checkpointC = torch.load(os.path.join(model_dir, 'Multi_LSTM-di-20-0.pth'))
checkpointD = torch.load(os.path.join(model_dir, 'Multi_LSTM-tri-20-0.pth'))

multi_nfr_model.load_state_dict(checkpointA['multi_lstm'])
multi_mono_model.load_state_dict(checkpointB['multi_lstm'])
multi_di_model.load_state_dict(checkpointC['multi_lstm'])
multi_tri_model.load_state_dict(checkpointD['multi_lstm'])

<All keys matched successfully>

In [ ]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps, n_features, to_do):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix + 1 > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :n_features], sequences[end_ix, -mapping[to_do]]
        X.append(seq_x)
        y.append(seq_y)

        # print(X)
        # print(y)

        # break
    return np.array(X), np.array(y)

###Load CSV file on which to predict


###Predict values for each of the columns

In [ ]:
path = "/content/drive/My Drive/project/"
dataset = pd.read_csv(path + 'split-29.csv')
dataset = dataset.to_numpy()

model_list = [multi_nfr_model, multi_mono_model, multi_di_model, multi_tri_model]
col_list = ['nfr', 'mono', 'di', 'tri']
final_data = []

for i in range(len(col_list)):
  to_do = col_list[i]
  mvnet = model_list[i]

  X_pred, y_pred = split_sequences(dataset, n_timesteps, n_features, to_do)

  X_pred = torch.from_numpy(X_pred).type(torch.Tensor)
  y_pred = torch.from_numpy(y_pred).type(torch.Tensor)

  pred = torch.utils.data.TensorDataset(X_pred,y_pred)

  pred_loader = torch.utils.data.DataLoader(dataset=pred, 
                                            batch_size=batch_size, 
                                            shuffle=False)

  print(X_pred.shape)
# predictions = price.iloc[0:lookback]

  predictions = []
  total_loss = 0

  with torch.no_grad():

    for x, y in pred_loader:

            x_batch, y_batch = x.cuda(), y.cuda()

            mv_net.init_hidden(x_batch.size(0))

            output = mv_net(x_batch) 
            loss = criterion(output.view(-1), y_batch)  

            predictions.extend(output.view(-1).cpu().numpy())
            # break

            total_loss += loss.item()


  print(total_loss)


  temp = y_pred.flatten().numpy()[0:look_back]
  predictions = np.append(temp, predictions)

  final_data.append(predictions)
  print(final_data)
  print(len(final_data[0]))

torch.Size([999979, 20, 4])
458374.8463915407
[array([6.5921   , 6.5921   , 6.5921   , ..., 1.3147007, 1.2994127,
       1.2938184], dtype=float32)]
999999
torch.Size([999979, 20, 4])
988384.0513844554
[array([6.5921   , 6.5921   , 6.5921   , ..., 1.3147007, 1.2994127,
       1.2938184], dtype=float32), array([0.       , 0.       , 0.       , ..., 1.3147007, 1.2994127,
       1.2938184], dtype=float32)]
999999
torch.Size([999979, 20, 4])
705339.2026189344
[array([6.5921   , 6.5921   , 6.5921   , ..., 1.3147007, 1.2994127,
       1.2938184], dtype=float32), array([0.       , 0.       , 0.       , ..., 1.3147007, 1.2994127,
       1.2938184], dtype=float32), array([9.9999997e-05, 9.9999997e-05, 9.9999997e-05, ..., 1.3147007e+00,
       1.2994127e+00, 1.2938184e+00], dtype=float32)]
999999
torch.Size([999979, 20, 4])
71768.68248007144
[array([6.5921   , 6.5921   , 6.5921   , ..., 1.3147007, 1.2994127,
       1.2938184], dtype=float32), array([0.       , 0.       , 0.       , ..., 1.314700

In [ ]:
temp = []

for i in range(len(final_data[0])):
  temp.append([final_data[0][i],final_data[1][i],final_data[2][i],final_data[3][i]])

df = pd.DataFrame(temp, columns=['nfr', 'mono', 'di', 'tri'])
print(df)

             nfr      mono        di       tri
0       6.592100  0.000000  0.000100  1.137200
1       6.592100  0.000000  0.000100  1.137200
2       6.592100  0.000000  0.000100  1.137200
3       6.592100  0.000000  0.000100  1.137200
4       6.592100  0.000000  0.000100  1.137200
...          ...       ...       ...       ...
999994  1.303105  1.303105  1.303105  1.303105
999995  1.299840  1.299840  1.299840  1.299840
999996  1.314701  1.314701  1.314701  1.314701
999997  1.299413  1.299413  1.299413  1.299413
999998  1.293818  1.293818  1.293818  1.293818

[999999 rows x 4 columns]


###Run K-Means to assign State Labels for each row

In [ ]:
n_samples, n_features = df.shape
n_classes = 3
estimator = KMeans(init='k-means++', n_clusters=n_classes, n_init=10)
estimator.fit(df)
labels = estimator.predict(df)
labels = pd.DataFrame(labels)
print(estimator.cluster_centers_)
nfr_values = np.array([estimator.cluster_centers_[i][0] for i in range(3)])
index_C = np.argmax(nfr_values)
index_B = np.argmin(nfr_values)
index_L = 3 - index_C - index_B
print(nfr_values)
print(index_L)
print(index_B)
print(index_C)
labels = labels.replace({index_C:'C', index_B:'B', index_L:'L'})
# labels = pd.get_dummies(labels[labels.columns[0]])
labels.head()

[[4.97485000e+05 4.42574661e+00 4.42574661e+00 4.42574661e+00
  4.42574661e+00]
 [8.32075000e+05 4.66232447e+00 4.66232447e+00 4.66232447e+00
  4.66232447e+00]
 [1.65409000e+05 4.84940970e+00 4.84901116e+00 4.84901117e+00
  4.84907991e+00]]
[497485. 832075. 165409.]
0
2
1


,0
0,B
1,B
2,B
3,B
4,B


###Save the Regresion and Label Prediction Files to the drive

In [ ]:
df.to_csv('multi_pred.csv')
!cp multi_pred.csv "drive/My Drive/"

labels.to_csv('multi_labels.csv')
!cp multi_labels.csv "drive/My Drive/"